## Setup

In [1]:
# Setup
import geopandas as gpd
import flopy
import numpy as np
import os
import pandas as pd

In [2]:
cwd = '\\'.join(os.getcwd().split('\\')[:-2])
cwd

'c:\\repos\\hm\\MAURICE'

## Tests

### Test: Load the model

In [3]:
# bug detected:
# 1. if the script is not in the same folder as the .nam file,
#    an error arises saying that the ._kx file is not found
# 2. if the .sfr file is in a SFR2 format (e.g. NSRTM = 2),
#    the file doesn't get read

m = flopy.mfusg.MfUsg.load(os.path.join(os.getcwd(), 't_m11_C1.nam'))

c:\Users\user\anaconda3\envs\maurice\Lib\site-packages\flopy\mbase.py:97: UserWarning: The program mfusg does not exist or is not executable.
  warn(


In [5]:
m.sfr.reach_data

rec.array([(4552, 0, 0, 0, 1,  1, 106., 0., 0., 0., 0., 0., 0., 0., 0.,   1,   2),
           (4553, 0, 0, 0, 1,  2,   8., 0., 0., 0., 0., 0., 0., 0., 0.,   2,   3),
           (4676, 0, 0, 0, 1,  3,  99., 0., 0., 0., 0., 0., 0., 0., 0.,   3,   4),
           (4677, 0, 0, 0, 1,  4,  63., 0., 0., 0., 0., 0., 0., 0., 0.,   4,   5),
           (4678, 0, 0, 0, 2,  1,  99., 0., 0., 0., 0., 0., 0., 0., 0.,   5,   6),
           (4679, 0, 0, 0, 2,  2,  97., 0., 0., 0., 0., 0., 0., 0., 0.,   6,   7),
           (4802, 0, 0, 0, 2,  3,   4., 0., 0., 0., 0., 0., 0., 0., 0.,   7,   8),
           (4803, 0, 0, 0, 2,  4, 100., 0., 0., 0., 0., 0., 0., 0., 0.,   8,   9),
           (4804, 0, 0, 0, 2,  5, 106., 0., 0., 0., 0., 0., 0., 0., 0.,   9,  10),
           (4805, 0, 0, 0, 2,  6,  88., 0., 0., 0., 0., 0., 0., 0., 0.,  10,  11),
           (4928, 0, 0, 0, 2,  7,  32., 0., 0., 0., 0., 0., 0., 0., 0.,  11,  12),
           (4929, 0, 0, 0, 2,  8, 122., 0., 0., 0., 0., 0., 0., 0., 0.,  12,  13),
    

In [18]:
# m.sfr.reach_data

In [33]:
# m.sfr.segment_data

In [ ]:
sp = 1 #stress period number
m.sfr.segment_data[sp-1]

In [12]:
m.sfr.nstrm

np.int64(174)

In [14]:
m.sfr.isfropt

0

In [ ]:
m.sfr.isfropt = 2
#setting it from here doesn't change the output format

#### Test: load sfr condition from shapefile

In [109]:
sfr_shp = gpd.read_file(os.path.join(cwd, 'test_models', 'SFR_ICALC2.shp'))

In [8]:
sfr_shp.Slope

0      0.000019
1      0.000019
2      0.000019
3      0.000019
4      0.000019
         ...   
169    0.000019
170    0.000019
171    0.000019
172    0.000019
173    0.000019
Name: Slope, Length: 174, dtype: float64

### Test: Modify the segment data parameters

In [29]:
# Load the data to be set up into the different stress periods
q = pd.read_csv(os.path.join(cwd, 'test_models', 'portate_segmenti.csv'))
q[q.isna()] = 0

In [30]:
for sp in range(1, 91+1):
    tool = q.loc[q.SP == sp, ['Qin', 'Sg_2', 'Sg_3', 'Sg_4', 'Sg_5', 'Sg_6', 'Sg_7']].values[0]
    tool = np.around(tool, 5)
    tool = [tool[0]] + [-x if x != 0 else 0 for x in tool[1:]]
    m.sfr.segment_data[sp-1].flow = tool

AttributeError: 'NoneType' object has no attribute 'segment_data'

### Test: write the new .sfr file

In [16]:
m.model_ws = os.path.join(cwd, 'test_models', 'tool_m11_c1', 'test_datasets_m11_c1')

In [19]:
m.write_input()

## Test: load reach and segment data from Excel file

In [3]:
# Load a tool model (with SFR package in a format readable by flopy)
m = flopy.mfusg.MfUsg.load(os.path.join(os.getcwd(), 't_m11_C1.nam'))

OSError: cannot find name file: c:\repos\hm\MAURICE\test_models\tool_m11_c1\t_m11_C1.nam

In [186]:
m.sfr


Streamflow-Routing (SFR2) Package Class

Parameters
----------
model : model object
    The model object (of type :class:'flopy.modflow.mf.Modflow') to which
    this package will be added.
nstrm : integer
    An integer value that can be specified to be positive or negative. The
    absolute value of NSTRM is equal to the number of stream reaches
    (finite-difference cells) that are active during the simulation and
    the number of lines of data to be included in Item 2, described below.
    When NSTRM is specified to be a negative integer, it is also used as a
    flag for changing the format of the data input, for simulating
    unsaturated flow beneath streams, and (or) for simulating transient
    streamflow routing (for MODFLOW-2005 simulations only), depending
    on the values specified for variables ISFROPT and IRTFLG, as described
    below. When NSTRM is negative, NSFRPAR must be set to zero, which means
    that parameters cannot be specified. By default, nstrm is set t

https://flopy.readthedocs.io/en/stable/Notebooks/sfrpackage_example.html

In [187]:
m.sfr.isfropt = 1
m.sfr.nrstrm = -174

In [188]:
m.sfr.write_file('test_v1.sfr')

## Continue

https://flopy.readthedocs.io/en/3.4.2/Notebooks/sfrpackage_example.html#define-dataset-6e-(channel-flow-data)-for-segment-1

In [81]:
# Load SFR characteristics from Excel files and shapefile
sfr_data = os.path.join(cwd, 'test_models', 'm11_C1_sfr_data.xlsx')
sfr_shp = gpd.read_file(os.path.join(cwd, 'test_models', 'SFR_ICALC2.shp'))

# Load general parameters (item 1)
it1 = pd.read_excel(sfr_data, sheet_name = 'ITEM1')

# Load reach data (item 2)
reach_data = pd.read_excel(sfr_data, sheet_name = 'ITEM2b')
reach_data = reach_data.merge(sfr_shp.loc[:, ['node', 'row', 'column', 'layer']], left_on = 'NRCH', right_on = 'node')
reach_data = reach_data.drop(columns = 'NRCH')
reach_data = reach_data.apply(pd.to_numeric)
reach_data.columns = ['iseg', 'ireach', 'rchlen', 'strtop', 'slope',  'strthick',  'strhc1', 'node', 'k', 'i', 'j']
reach_data = reach_data.loc[:,:].to_records(index = False)
# flopy adds 1 to node???
# reach_data.node = reach_data.node - 1

# Load item 5
it5 = pd.read_excel(sfr_data, sheet_name = 'ITEM5')

# Load segment data (item 6a)
segment_data = pd.read_excel(sfr_data, sheet_name = 'ITEM6abc')
segment_data.columns = [x.lower() for x in segment_data.columns]
segment_data = segment_data.loc[:,:].to_records(index = False)
segment_data = {0: segment_data}

it6d = pd.read_excel(sfr_data, sheet_name = 'ITEM6d')
geom_data = {}
for seg in it6d.segment.unique():
    tool = it6d.loc[it6d.segment == seg, [f'v{i}' for i in range(1,9)]].to_numpy().copy()
    geom_data[int(seg)] = [tool[0].tolist(), tool[1].tolist()]
geom_data = {0: geom_data}

# Set the unit number
unit_number = 27

# Load the data to be set up into the different stress periods
q = pd.read_csv(os.path.join(cwd, 'test_models', 'portate_segmenti.csv'))
q[q.isna()] = 0

In [82]:
d_5 = {}
for i in range(0,91):
    d_5[i] = it5.values[0].tolist()

In [88]:
sfr.nss

7

In [ ]:
m = flopy.modflow.Modflow(stress_periods = 91)
sfr = flopy.modflow.ModflowSfr2(
    m,
    nstrm = it1.NSTRM.values[0],              # number of reaches
    nss = it1.NSS.values[0],                  # number of segments
    const = it1.CONST.values[0],              # constant for manning's equation: 1 for m/s
    dleak = it1.DLEAK.values[0],              # closure tolerance for stream stage computation
    ipakcb = it1.ISTCB1.values[0],            # flag for writing SFR output to cell-by-cell budget (on unit 50)
    istcb2 = it1.ISTCB2.values[0],            # flag for writing SFR output to text file
    dataset_5 = d_5,
    unit_number = unit_number,
    isfropt = it1.ISFROPT.values[0],          # ISFROPT parameter, defines input file format writing
    reach_data = reach_data,

    irtflg = 1, # activate transient streamflow routing
    channel_geometry_data=geom_data,

    segment_data = segment_data

    # nsfr
)

c:\Users\user\anaconda3\envs\maurice\Lib\site-packages\flopy\mbase.py:442: UserWarning: unhandled keywords: {'stress_periods': 91}
  warn(


In [84]:
segment_data

{0: rec.array([(1, 2, 2, 0, 0, 0,  1.185, 0, 0, 0, 0.017, 0.017, 19.73, 19.73),
            (2, 2, 3, 0, 0, 0,  0.   , 0, 0, 0, 0.017, 0.017, 19.73, 19.73),
            (3, 2, 4, 0, 0, 0, -0.01 , 0, 0, 0, 0.017, 0.017, 19.73, 19.73),
            (4, 2, 5, 0, 0, 0,  0.   , 0, 0, 0, 0.017, 0.017, 19.73, 19.73),
            (5, 2, 6, 0, 0, 0,  0.   , 0, 0, 0, 0.017, 0.017, 19.73, 19.73),
            (6, 2, 7, 0, 0, 0,  0.   , 0, 0, 0, 0.017, 0.017, 19.73, 19.73),
            (7, 2, 0, 0, 0, 0,  0.   , 0, 0, 0, 0.017, 0.017, 19.73, 19.73)],
           dtype=[('nseg', '<i8'), ('icalc', '<i8'), ('outseg', '<i8'), ('iupseg', '<i8'), ('iprior', '<i8'), ('nstrpts', '<i8'), ('flow', '<f8'), ('runoff', '<i8'), ('etsw', '<i8'), ('pptsw', '<i8'), ('roughch', '<f8'), ('roughbk', '<f8'), ('width1', '<f8'), ('width2', '<f8')])}

In [85]:
# Insert the necessary flow values for each stress period
for sp in range(1, 91+1):
    tool = q.loc[q.SP == sp, ['Qin', 'Sg_2', 'Sg_3', 'Sg_4', 'Sg_5', 'Sg_6', 'Sg_7']].values[0]
    tool = np.around(tool, 5)
    tool = [tool[0]] + [-x if x != 0 else 0 for x in tool[1:]]
    segment_data[0].flow = tool
    sfr.segment_data[sp-1] = segment_data[0].copy()
    # sfr.stress_period_data[sp-1] = segment_data[0].copy()

In [86]:
# non stampa gli stress periods
sfr.write_file('test_seg.sfr')

KeyError: 'cdpth'

In [41]:
sfr.segment_data

{0: rec.array([(1, 1, 2, 0, 0, 0, 1.18571, 0, 0, 0, 0.017, 0.017, 0, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
            (2, 1, 3, 0, 0, 0, 0.     , 0, 0, 0, 0.017, 0.017, 0, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
            (3, 1, 4, 0, 0, 0, 0.     , 0, 0, 0, 0.017, 0.017, 0, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
            (4, 1, 5, 0, 0, 0, 0.     , 0, 0, 0, 0.017, 0.017, 0, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
            (5, 1, 6, 0, 0, 0, 0.     , 0, 0, 0, 0.017, 0.017, 0, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
            (6, 1, 7, 0, 0, 0, 0.     , 0, 0, 0, 0.017, 0.017, 0, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
            (7, 1, 0, 0, 0, 0, 0.     , 0, 0, 0, 0.017, 0.017, 0, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)],
           dtype=[('nseg', '<i8'), ('icalc', '<i8')

In [11]:
sfr.get_variable_by_stress_period('flow')

rec.array([(38, 2, 1, 1, 1, 1.18571)],
          dtype=[('k', '<i8'), ('i', '<i8'), ('j', '<i8'), ('iseg', '<i8'), ('ireach', '<i8'), ('flow0', '<f8')])

# Esempio SFR online

In [89]:
m = flopy.modflow.Modflow()

In [93]:
rpth = os.path.join(
    cwd, 'test_models',"flopy_sfr_examples", "test1ss_reach_data.csv"
)
reach_data = np.genfromtxt(rpth, delimiter=",", names=True)
reach_data

array([(0.,  0., 0., 1., 1., 4500.), (0.,  1., 1., 1., 2., 7000.),
       (0.,  2., 2., 1., 3., 6000.), (0.,  2., 3., 1., 4., 5550.),
       (0.,  3., 4., 2., 1., 6500.), (0.,  4., 5., 2., 2., 5000.),
       (0.,  5., 5., 2., 3., 5000.), (0.,  6., 5., 2., 4., 5000.),
       (0.,  7., 5., 2., 5., 5000.), (0.,  2., 4., 3., 1., 5000.),
       (0.,  2., 5., 3., 2., 5000.), (0.,  2., 6., 3., 3., 4500.),
       (0.,  3., 7., 3., 4., 6000.), (0.,  4., 7., 3., 5., 5000.),
       (0.,  5., 7., 3., 6., 2000.), (0.,  4., 9., 4., 1., 2500.),
       (0.,  4., 8., 4., 2., 5000.), (0.,  5., 7., 4., 3., 3500.),
       (0.,  5., 7., 5., 1., 4000.), (0.,  6., 6., 5., 2., 5000.),
       (0.,  7., 6., 5., 3., 3500.), (0.,  7., 5., 5., 4., 2500.),
       (0.,  8., 5., 6., 1., 5000.), (0.,  9., 6., 6., 2., 5000.),
       (0., 10., 6., 6., 3., 5000.), (0., 11., 6., 6., 4., 5000.),
       (0., 12., 6., 6., 5., 2000.), (0., 13., 8., 7., 1., 5000.),
       (0., 12., 7., 7., 2., 5500.), (0., 12., 6., 7., 3., 500

In [95]:
spth = os.path.join(
   cwd, 'test_models',"flopy_sfr_examples", "test1ss_segment_data.csv"
)
ss_segment_data = np.genfromtxt(spth, delimiter=",", names=True)
segment_data = {0: ss_segment_data}
segment_data[0][0:1]["width1"]

array([0.])

In [104]:
segment_data[0]['flow']

array([ 25.,   0.,  10.,  10.,   0.,   0., 150.,   0.])

In [96]:
channel_flow_data = {
    0: {
        1: [
            [0.5, 1.0, 2.0, 4.0, 7.0, 10.0, 20.0, 30.0, 50.0, 75.0, 100.0],
            [0.25, 0.4, 0.55, 0.7, 0.8, 0.9, 1.1, 1.25, 1.4, 1.7, 2.6],
            [3.0, 3.5, 4.2, 5.3, 7.0, 8.5, 12.0, 14.0, 17.0, 20.0, 22.0],
        ]
    }
}

In [97]:
channel_geometry_data = {
    0: {
        7: [
            [0.0, 10.0, 80.0, 100.0, 150.0, 170.0, 240.0, 250.0],
            [20.0, 13.0, 10.0, 2.0, 0.0, 10.0, 13.0, 20.0],
        ],
        8: [
            [0.0, 10.0, 80.0, 100.0, 150.0, 170.0, 240.0, 250.0],
            [25.0, 17.0, 13.0, 4.0, 0.0, 10.0, 16.0, 20.0],
        ],
    }
}

In [ ]:
nstrm = len(reach_data)  # number of reaches
nss = len(segment_data[0])  # number of segments
nsfrpar = 0  # number of parameters (not supported)
nparseg = 0
const = 1.486  # constant for manning's equation, units of cfs
dleak = 0.0001  # closure tolerance for stream stage computation
ipakcb = 53  # flag for writing SFR output to cell-by-cell budget (on unit 53)
istcb2 = 81  # flag for writing SFR output to text file
dataset_5 = {0: [nss, 0, 0]}  # dataset 5 (see online guide)